In [1]:
import pandas as pd
from pandas import json_normalize

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

import requests 
import json
import os
from dotenv import load_dotenv

from pymongo import MongoClient
import pandas as pd


import src.nearby_function as nearby



In [2]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("Crunchbase")

In [3]:
# filter with regex "desing", 'total_money_raised', "offices[0].country_code"

In [4]:
#DESIGN or tech companies nearby
filter_10 = {"description": {"$regex": "design"}}
filter_11 = {"description": {"$regex": "technology"}}
filter_12 = {"description": {"$regex": "software"}}

filter_2 = {"tag_list": {"$regex": "design"}}

filter_3 = {"name": {"$regex": "design"}}

filter_40 = {"category_code": {"$regex": "design"}}
filter_41 = {"category_code": {"$regex": "games_video"}}
filter_42 = {"category_code": {"$regex": "web"}}
filter_43 = {"category_code": {"$regex": "mobile"}}
filter_44 = {"category_code": {"$regex": "photo_video"}}

#Near to SUCCESSFUL tech startups
filter_50 = {"total_money_raised": {"$regex": "M"}}
filter_51 = {"total_money_raised": {"$regex": "B"}}
filter_52 = {"total_money_raised": {"$regex": "^k"}}
filter_53 = {"total_money_raised":{ "$not": {"$regex": "/$0/"}}}




all_conditions = [filter_10, filter_11, filter_12, filter_2, filter_3, filter_40, 
                  filter_41, filter_42, filter_43, filter_44, filter_50, filter_51, filter_52, filter_53]

In [5]:
projection = {"name": 1, 
              "number_of_employees": 1,
              "total_money_raised": 1,
              "offices.country_code": 1,
              "offices.city": 1,
              "offices.latitude": 1,
              "offices.longitude": 1,
              "_id":0}

result = list(c.find({"$or": all_conditions,
                      
                "number_of_employees": {"$lte": 150 } # STARTUP
                     
                     }, projection))

df = pd.DataFrame(result)

In [6]:
df

,name,number_of_employees,total_money_raised,offices
0,Wetpaint,47,$39.8M,"[{'city': 'Seattle', 'country_code': 'USA', 'l..."
1,Digg,60,$45M,"[{'city': 'San Francisco', 'country_code': 'US..."
2,Geni,18,$16.5M,"[{'city': 'West Hollywood', 'country_code': 'U..."
3,Fox Interactive Media,0,$0,"[{'city': 'Beverly Hills', 'country_code': 'US..."
4,Scribd,50,$25.8M,"[{'city': 'San Francisco', 'country_code': 'US..."
...,...,...,...,...
8130,OfficialVirtualDJ,102,€25M,"[{'city': 'London', 'country_code': 'GBR', 'la..."
8131,Eazeeloans,25,$0,"[{'city': 'Mumbai', 'country_code': 'IND', 'la..."
8132,DocASAP,7,$800k,"[{'city': 'New York', 'country_code': 'USA', '..."
8133,cotralis,1,$0,[]


In [7]:
col_to_extract = df['offices']
col_to_extract[0][0]

{'city': 'Seattle',
 'country_code': 'USA',
 'latitude': 47.603122,
 'longitude': -122.333253}

In [8]:
def extraction(col, new_col, data):    
    list_ = []
    for i in range(len(col)):
        try:           
            list_.append(col[i][0][data])             
        except:
            list_.append("null")
    df[new_col]= list_
    return

In [9]:
extraction(col_to_extract, "Country", "country_code")
extraction(col_to_extract, "City", "city")
extraction(col_to_extract, "lat", "latitude")
extraction(col_to_extract, "lon", "longitude")

In [10]:
df

,name,number_of_employees,total_money_raised,offices,Country,City,lat,lon
0,Wetpaint,47,$39.8M,"[{'city': 'Seattle', 'country_code': 'USA', 'l...",USA,Seattle,47.603122,-122.333253
1,Digg,60,$45M,"[{'city': 'San Francisco', 'country_code': 'US...",USA,San Francisco,37.764726,-122.394523
2,Geni,18,$16.5M,"[{'city': 'West Hollywood', 'country_code': 'U...",USA,West Hollywood,34.090368,-118.393064
3,Fox Interactive Media,0,$0,"[{'city': 'Beverly Hills', 'country_code': 'US...",USA,Beverly Hills,34.076179,-118.39417
4,Scribd,50,$25.8M,"[{'city': 'San Francisco', 'country_code': 'US...",USA,San Francisco,37.789634,-122.404052
...,...,...,...,...,...,...,...,...
8130,OfficialVirtualDJ,102,€25M,"[{'city': 'London', 'country_code': 'GBR', 'la...",GBR,London,None,None
8131,Eazeeloans,25,$0,"[{'city': 'Mumbai', 'country_code': 'IND', 'la...",IND,Mumbai,None,None
8132,DocASAP,7,$800k,"[{'city': 'New York', 'country_code': 'USA', '...",USA,New York,None,None
8133,cotralis,1,$0,[],null,null,null,null


In [11]:
df.drop(['offices'],axis =1, inplace=True)
df.dropna(inplace=True)
df = df[df["Country"] != "null"]

In [12]:
df

,name,number_of_employees,total_money_raised,Country,City,lat,lon
0,Wetpaint,47,$39.8M,USA,Seattle,47.603122,-122.333253
1,Digg,60,$45M,USA,San Francisco,37.764726,-122.394523
2,Geni,18,$16.5M,USA,West Hollywood,34.090368,-118.393064
3,Fox Interactive Media,0,$0,USA,Beverly Hills,34.076179,-118.39417
4,Scribd,50,$25.8M,USA,San Francisco,37.789634,-122.404052
...,...,...,...,...,...,...,...
8123,InstantQ,3,$0,USA,Sunnyvale,37.369195,-122.036849
8125,NextLabs,100,$0,USA,San Mateo,37.553121,-122.301534
8127,acrossair,6,$0,GBR,London,51.517742,-0.116537
8128,MyChances,2,$0,USA,Reno,39.394564,-119.812079


In [187]:
df["Country"].value_counts()[:1]

USA    3437
Name: Country, dtype: int64

In [188]:
df['City'].where(df['Country'] == 'USA').value_counts()[:3]

San Francisco    320
New York         249
Seattle           89
Name: City, dtype: int64

In [189]:
# df_SanF = df[df['City'] == 'San Francisco']
df_SanF = df[df['City'] == 'San Francisco']

# TubeSpree, Bizroof
# FeVote
# Lending Karma
# ARX
# Ecolect

In [190]:
df_SanF

,name,number_of_employees,total_money_raised,Country,City,lat,lon
1,Digg,60,$45M,USA,San Francisco,37.764726,-122.394523
4,Scribd,50,$25.8M,USA,San Francisco,37.789634,-122.404052
8,Powerset,60,$22.5M,USA,San Francisco,37.778613,-122.395289
9,Technorati,35,$32.1M,USA,San Francisco,37.779558,-122.393041
12,Kyte,40,$23.4M,USA,San Francisco,37.788482,-122.409173
...,...,...,...,...,...,...,...
7776,UTOPY,40,$7.5M,USA,San Francisco,37.767575,-122.411157
7821,ZoomSystems,100,$87M,USA,San Francisco,37.781689,-122.391061
7879,Mondo Media,9,$0,USA,San Francisco,37.764169,-122.402021
7902,TwtBks,3,$25k,USA,San Francisco,37.761102,-122.416001


In [16]:
df

,name,number_of_employees,total_money_raised,Country,City,lat,lon
0,Wetpaint,47,$39.8M,USA,Seattle,47.603122,-122.333253
1,Digg,60,$45M,USA,San Francisco,37.764726,-122.394523
2,Geni,18,$16.5M,USA,West Hollywood,34.090368,-118.393064
3,Fox Interactive Media,0,$0,USA,Beverly Hills,34.076179,-118.39417
4,Scribd,50,$25.8M,USA,San Francisco,37.789634,-122.404052
...,...,...,...,...,...,...,...
8123,InstantQ,3,$0,USA,Sunnyvale,37.369195,-122.036849
8125,NextLabs,100,$0,USA,San Mateo,37.553121,-122.301534
8127,acrossair,6,$0,GBR,London,51.517742,-0.116537
8128,MyChances,2,$0,USA,Reno,39.394564,-119.812079


In [391]:
import math


def haversine(coord1, coord2):

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    km = round(km, 3)
    meters = round(meters, 3)
    return meters
    


In [ ]:
'''STARTING THE PROJECT

:uno: Explore database & filter it

:dos: Explore how to query (no matter city yet):

Create small function to query starbucks (limit=1, city=whichever)
Small function to query something else (limit=1, city=whichever)
Maybe these two can be the same and it’s just a matter of parametrization?
and so on :cara_ligeramente_sonriente:
:tres: Focus on getting one thing at a time: you’ll scale things up later on
It’s about making decisions. Try to justify them with data as much as you can, but it’s okay as some of them you’ll have to make a bit randomly.
Goal of the project is:
Querying a database
Extracting information through an API
Loading that information back into a database
Aggregate and explore by: doing geoqueries
Visualize using maps :mapamundi:'''

In [18]:
data = {'Requirements ':   ['Design companies nearby', 
                            '30% of staff have childs /parks and schools nearby/',
                            'Near successful tech startups',
                            'Starbucks not too far',
                            'Account managers need to travel a lot',
                            'Place to go party',
                            'The CEO is vegan',
                            'A basketball stadium must be around 10 Km',
                            'Dog hairdresser not too far'],
        
        'Priorization Factor': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
  


df_info = pd.DataFrame(data)
df_info                      

,Requirements,Priorization Factor
0,Design companies nearby,1
1,30% of staff have childs /parks and schools ne...,1
2,Near successful tech startups,1
3,Starbucks not too far,1
4,Account managers need to travel a lot,1
5,Place to go party,1
6,The CEO is vegan,1
7,A basketball stadium must be around 10 Km,1
8,Dog hairdresser not too far,1


In [19]:
load_dotenv()

True

In [20]:
token_for_foursquare = os.getenv("API_Key")

In [21]:
def get_results_from_foursquare (query, location, limit=1):
    
    ll = f"{location[0]}%2C{location[1]}"
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&limit={str(limit)}"

    headers = {
        "accept": "application/json",
        "Authorization": token_for_foursquare,
    }
    
    response = requests.get(url, headers=headers).json()
    
    return response

In [22]:
sanFran_location=[37.7676672,-122.4461183] 
sf_lat = 37.7676672
sf_long = -122.4461183

In [23]:
sanFran_map = Map(location = [sf_lat, sf_long], zoom_start = 13)
sanFran_map

# companies

In [24]:
#1. MARKER without icon
for index, row in df_SanF.iterrows():
    parks = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}

#2. Icon
    icon = Icon (color="blue", opacity = 0.6, prefix = "fa", icon="briefcase", icon_color = "black")    
    
#3. Marker
    new_marker = Marker(**parks, icon = icon, radius = 2)

#4. Add the Marker
    new_marker.add_to(sanFran_map)

In [25]:
companies_group = folium.FeatureGroup(name=f"company: {df_SanF.shape[1]}")

In [26]:
sanFran_map_HM = Map(location = [sf_lat, sf_long], zoom_start = 13)
HeatMap(data=df_SanF[["lat", "lon"]], 
        radius=25,
       gradient = {0.4: 'blue', 0.65: 'lightblue', 1: 'lightblue'}).add_to(companies_group)

In [27]:
companies_group.add_to(sanFran_map_HM)

In [28]:
sanFran_map_HM

In [29]:
#1. MARKER without icon
for index, row in df_SanF.iterrows():
    parks = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}

#2. Icon
    icon = Icon (color="blue", opacity = 0.01, prefix = "fa", icon="briefcase", icon_color = "black")    
    
#3. Marker
    new_marker = Marker(**parks, icon = icon, radius = 0.5)

#4. Add the Marker
    new_marker.add_to(sanFran_map_HM)

In [131]:
sanFran_map_HM

In [156]:
sanFran_map_HM

# QUERY = Airports

In [31]:
location_SFO=[37.6399899,-122.3942075]

lat_SFO = 37.639989
lon_SFO = -122.3942075

In [33]:
icon_SFO = Icon(color = "lightred",
            opacity = 0.1,
            prefix = "fa",
            icon = "plane",
            icon_color = "black"
)

In [34]:
airport = Marker(location = [lat_SFO, lon_SFO], tooltip="Airport SFO", icon=icon_SFO)

In [39]:
airport.add_to(sanFran_map_HM)

In [41]:
airport.add_to(sanFran_map)

In [42]:
sanFran_map

In [43]:
bart_metro = [{'station': 'bart_stc_1',
  'lat': 37.789364,
  'lon': -122.4035257},
  
  {'station': 'bart_stc_2',
  'lat': 37.7853959,
  'lon': -122.4076993},
   
   {'station': 'bart_stc_3',
  'lat': 37.7796468,
  'lon': -122.4132246},
    
    {'station': 'bart_stc_4',
  'lat':37.7652648,
  'lon': -122.4195441},
     
    {'station': 'bart_stc_5',
  'lat': 37.7512276,
  'lon': -122.417922},
          
   {'station': 'bart_stc_6',
  'lat':37.7340611,
  'lon': -122.432054},
    
    {'station': 'bart_stc_7',
  'lat':37.705859,
  'lon': -122.4699171}]

In [44]:
df_metro = pd.DataFrame(bart_metro)
df_metro

,station,lat,lon
0,bart_stc_1,37.789364,-122.403526
1,bart_stc_2,37.785396,-122.407699
2,bart_stc_3,37.779647,-122.413225
3,bart_stc_4,37.765265,-122.419544
4,bart_stc_5,37.751228,-122.417922
5,bart_stc_6,37.734061,-122.432054
6,bart_stc_7,37.705859,-122.469917


In [51]:
#1. MARKER without icon
for index, row in df_metro.iterrows():
    station = {"location": [row["lat"], row["lon"]], "tooltip": row["station"]}

#2. Icon
    icon = Icon (color="lightred", opacity = 0.6, prefix = "fa", icon="subway", icon_color = "black")    
    
#3. Marker
    new_marker = Marker(**station, icon = icon, radius = 2)

#4. Add the Marker
    new_marker.add_to(sanFran_map)

In [52]:
sanFran_map

# QUERY = PARK

In [88]:
query_10 = "park"

In [89]:
sanFran_location=[37.7676672,-122.4461183] 
sf_lat = 37.7676672
sf_long = -122.4461183


In [90]:
response_park = get_results_from_foursquare(query_10,sanFran_location)

In [91]:
latitude = response_park["results"][0]["geocodes"]["main"]["latitude"]
longitude = response_park["results"][0]["geocodes"]["main"]["longitude"]
address = response_park["results"][0]["location"]["address"]
name = response_park["results"][0]["name"]
distance = response_park["results"][0]['distance']
coord = [latitude, longitude]

print(distance)
print(coord)
print(address)
print(name)

704
[37.764875, -122.438974]
Roosevelt Way
Corona Heights Park


In [92]:
# Doing the same thing with limit=20

all_parks = get_results_from_foursquare (query_10, sanFran_location, limit=20)

In [94]:
new_list_park = []
for i in all_parks["results"]:
    
    name = i["name"]
    address = i["location"]["formatted_address"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
    distance = i['distance']
    
    
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_park.append({"name":name, "lat":lat, "lon":lon, "type":type_, "distance":distance})
    
new_list_park[:1]

[{'name': 'Tank Hill Park',
  'lat': 37.759904,
  'lon': -122.447564,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.759904, -122.447564]}},
  'distance': 865}]

In [95]:
len(new_list_park)

20

In [96]:
df1= pd.DataFrame(new_list_park)
df1["name"] = "park"
df1

,name,lat,lon,type,distance
0,park,37.759904,-122.447564,"{'typepoint': {'type': 'Point', 'coordinates':...",865
1,park,37.769190,-122.438083,"{'typepoint': {'type': 'Point', 'coordinates':...",725
2,park,37.769800,-122.433532,"{'typepoint': {'type': 'Point', 'coordinates':...",1175
3,park,37.756238,-122.471921,"{'typepoint': {'type': 'Point', 'coordinates':...",2589
4,park,37.750928,-122.468739,"{'typepoint': {'type': 'Point', 'coordinates':...",2763
5,park,37.767875,-122.441499,"{'typepoint': {'type': 'Point', 'coordinates':...",406
6,park,37.762666,-122.451597,"{'typepoint': {'type': 'Point', 'coordinates':...",721
7,park,37.768082,-122.457652,"{'typepoint': {'type': 'Point', 'coordinates':...",995
8,park,37.764875,-122.438974,"{'typepoint': {'type': 'Point', 'coordinates':...",704
9,park,37.758206,-122.441588,"{'typepoint': {'type': 'Point', 'coordinates':...",1120


In [97]:
#1. MARKER without icon
for index, row in df1.iterrows():
    parks = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}

#2. Icon
    icon = Icon (color="green", opacity = 0.6, prefix = "fa", icon="tree", icon_color = "black")    
    
#3. Marker
    new_marker = Marker(**parks, icon = icon, radius = 2)

#4. Add the Marker
    new_marker.add_to(sanFran_map)

In [98]:
sanFran_map

In [370]:
park = db.get_collection("park")
park

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'park')

In [371]:
park.insert_many(new_list_school)

In [369]:
park.find_one()

{'_id': ObjectId('635ff0d74f45b207b5cf1a1e'),
 'name': "Children's Day School",
 'lat': 37.764166,
 'lon': -122.425172,
 'type': {'typepoint': {'type': 'Point',
   'coordinates': [37.764166, -122.425172]}},
 'distance': 1890}

In [193]:
def type_point (list_):
    return {"type": "Point", "coordinates": list_}

In [327]:
list_nearest_park=[]
list_nearest_park_coor=[]

for index, rows in df_SanF.iterrows():
    try:
        lat =  df_SanF['lat'][index]
        lon = df_SanF['lon'][index]
    
        query = {"type.typepoint": {
                "$near": {
                    "$geometry": {'type': 'Point', 'coordinates':[lon, lat]}, "$maxDistance": 2000
                }}       
            }
        list_ = list(park.find(query))
        v1 = list_[0]['name']
        v2 = list_[0]['type']['typepoint']['coordinates']
        
        list_nearest_park.append(v1)
        list_nearest_park_coor.append(v2)
        
    except:
        lat = 0
        lon = 0
    
        query = {"type.typepoint": {
                "$near": {
                    "$geometry": {'type': 'Point', 'coordinates':[lon, lat]}, "$maxDistance": 2000
                }}       
            }
        
        
    

# QUERY = Schools

In [99]:
query_11 = "school"

In [104]:
response_school = get_results_from_foursquare(query_11,sanFran_location)

In [106]:
# Doing the same thing with limit=20

all_school = get_results_from_foursquare (query_11, sanFran_location, limit=20)

In [404]:
new_list_school = []
for i in all_school["results"]:
    
    name = i["name"]
    address = i["location"]["formatted_address"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
    distance = i['distance']
    
    
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lon, lat]}}
    
    new_list_school.append({"name":name, "lon":lon, "lat":lon, "type":type_, "distance":distance})
    
new_list_school[:2]

[{'name': "Children's Day School",
  'lon': -122.425172,
  'lat': -122.425172,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [-122.425172, 37.764166]}},
  'distance': 1890},
 {'name': 'The Scandinavian School In San Francisco',
  'lon': -122.458099,
  'lat': -122.458099,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [-122.458099, 37.746681]}},
  'distance': 2536}]

In [108]:
len(new_list_school)

20

In [109]:
df11= pd.DataFrame(new_list_school)
df11["name"] = "School"
df11

,name,lat,lon,type,distance
0,School,37.764166,-122.425172,"{'typepoint': {'type': 'Point', 'coordinates':...",1890
1,School,37.746681,-122.458099,"{'typepoint': {'type': 'Point', 'coordinates':...",2536
2,School,37.773306,-122.413743,"{'typepoint': {'type': 'Point', 'coordinates':...",2922
3,School,37.792933,-122.433469,"{'typepoint': {'type': 'Point', 'coordinates':...",3018
4,School,37.738550,-122.405458,"{'typepoint': {'type': 'Point', 'coordinates':...",4822
5,School,37.721607,-122.445007,"{'typepoint': {'type': 'Point', 'coordinates':...",5099
6,School,37.657254,-122.415280,"{'typepoint': {'type': 'Point', 'coordinates':...",12578
7,School,37.646130,-122.429944,"{'typepoint': {'type': 'Point', 'coordinates':...",13540
8,School,37.854105,-122.289566,"{'typepoint': {'type': 'Point', 'coordinates':...",16762
9,School,37.793312,-122.248663,"{'typepoint': {'type': 'Point', 'coordinates':...",17575


In [110]:
#1. MARKER without icon
for index, row in df11.iterrows():
    school = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}

#2. Icon
    icon = Icon (color="lightblue", opacity = 0.6, prefix = "fa", icon="book", icon_color = "black")    
    
#3. Marker
    new_marker = Marker(**school, icon = icon, radius = 2)

#4. Add the Marker
    new_marker.add_to(sanFran_map)

In [111]:
sanFran_map

In [367]:
school = db.get_collection("school")
school

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'school')

In [405]:
pa = db.get_collection("pa3")
pa.insert_many(new_list_school)

In [398]:
school.insert_many(new_list_school)

In [369]:
school.find_one()

{'_id': ObjectId('635ff0d74f45b207b5cf1a1e'),
 'name': "Children's Day School",
 'lat': 37.764166,
 'lon': -122.425172,
 'type': {'typepoint': {'type': 'Point',
   'coordinates': [37.764166, -122.425172]}},
 'distance': 1890}

In [193]:
def type_point (list_):
    return {"type": "Point", "coordinates": list_}

In [327]:
list_nearest_school=[]
list_nearest_school_coor=[]

for index, rows in df_SanF.iterrows():
    try:
        lat =  df_SanF['lat'][index]
        lon = df_SanF['lon'][index]
    
        query = {"type.typepoint": {
                "$near": {
                    "$geometry": {'type': 'Point', 'coordinates':[lon, lat]}, "$maxDistance": 2000
                }}       
            }
        list_ = list(school.find(query))
        v1 = list_[0]['name']
        v2 = list_[0]['type']['typepoint']['coordinates']
        
        list_nearest_school.append(v1)
        list_nearest_school_coor.append(v2)
        
    except:
        lat = 0
        lon = 0
    
        query = {"type.typepoint": {
                "$near": {
                    "$geometry": {'type': 'Point', 'coordinates':[lon, lat]}, "$maxDistance": 2000
                }}       
            }
        
        
    

# QUERY = vegan

In [112]:
query_21 = "vegan"

In [113]:
response_vegan = get_results_from_foursquare(query_21,sanFran_location)

In [114]:
# Doing the same thing with limit=20

all_vegan = get_results_from_foursquare (query_21, sanFran_location, limit=20)

In [167]:
new_list_vegan = []
for i in all_vegan["results"]:
    
    name = i["name"]
    address = i["location"]["formatted_address"]
    lon = i["geocodes"]["main"]["latitude"]
    lat = i["geocodes"]["main"]["longitude"]
    distance = i['distance']
    
    
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_vegan.append({"name":name, "lat":lat, "lon":lon, "type":type_, "distance":distance})
    
new_list_vegan[:2]

[{'name': 'Gracias Madre',
  'lat': -122.419145,
  'lon': 37.761621,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [-122.419145, 37.761621]}},
  'distance': 2467},
 {'name': 'Shizen',
  'lat': -122.421604,
  'lon': 37.768066,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [-122.421604, 37.768066]}},
  'distance': 2152}]

In [182]:
vegan = db.get_collection("vegan")
vegan

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'vegan')

In [183]:
vegan.insert_many(new_list_vegan)

In [184]:
vegan.find_one()

{'_id': ObjectId('635fc9e34f45b207b5cf1a0a'),
 'name': 'Gracias Madre',
 'lat': -122.419145,
 'lon': 37.761621,
 'type': {'typepoint': {'type': 'Point',
   'coordinates': [-122.419145, 37.761621]}},
 'distance': 2467}

In [193]:
def type_point (list_):
    return {"type": "Point", "coordinates": list_}

In [ ]:
converted_location una compañia 
df, cada row, cada compañia va a hacer una query para coger el sitio más cercano
con el sitio mas cercano a cada empresa -> dist entre empresa y sitio mas cercano

In [408]:
list_nearest_vegan=[]
list_nearest_vegan_coor=[]
dist=[]

for index, rows in df_SanF.iterrows():
    try:
        lat =  df_SanF['lat'][index]
        lon = df_SanF['lon'][index]
        
        coord1 = [lon, lat]
    
        query = {"type.typepoint": {
                "$near": {
                    "$geometry": {'type': 'Point', 'coordinates':[lon, lat]}, "$maxDistance": 2000
                }}       
            }
        list_ = list(vegan.find(query))
        v1 = list_[0]['name']
        coord2 = list_[0]['type']['typepoint']['coordinates']
        
        list_nearest_vegan.append(v1)
        list_nearest_vegan_coor.append(coord2)
        dist.append(haversine(coord1, coord2))
        
    except:
        lat = 0
        lon = 0
    
        query = {"type.typepoint": {
                "$near": {
                    "$geometry": {'type': 'Point', 'coordinates':[lon, lat]}, "$maxDistance": 2000
                }}       
            }
    

[-122.394523, 37.764726]
[-122.410305, 37.76857]
[-122.404052, 37.789634]
[-122.40348, 37.795567]
[-122.395289, 37.778613]
[-122.389344, 37.783824]
[-122.393041, 37.779558]
[-122.389344, 37.783824]
[-122.409173, 37.788482]
[-122.405173, 37.795717]
[-122.397224, 37.762541]
[-122.410305, 37.76857]
[-122.393913, 37.780716]
[-122.389344, 37.783824]
[-122.4038047, 37.7870842]
[-122.40348, 37.795567]
[-122.414689, 37.758113]
[-122.41851, 37.755723]
[-122.395184, 37.789268]
[-122.389344, 37.783824]
[-122.392144, 37.781427]
[-122.389344, 37.783824]
[-122.401116, 37.782103]
[-122.389344, 37.783824]
[-122.40184, 37.779026]
[-122.389344, 37.783824]
[-122.419204, 37.775196]
[-122.416164, 37.77789]
[-122.419204, 37.775196]
[-122.416164, 37.77789]
[-122.3985871, 37.7618242]
[-122.410305, 37.76857]
[-122.392861, 37.76785]
[-122.410305, 37.76857]
[-122.392142, 37.782263]
[-122.389344, 37.783824]
[-122.406676, 37.766909]
[-122.410305, 37.76857]
[-122.393041, 37.779558]
[-122.389344, 37.783824]
[-122.40

[-122.389344, 37.783824]
[-122.400346, 37.796843]
[-122.40348, 37.795567]
[-122.3943387, 37.7881481]
[-122.389344, 37.783824]
[-122.428193, 37.79289]
[-122.424833, 37.798627]
[-122.438231, 37.80204]
[-122.432295, 37.807002]
[-122.142326, 37.459112]
[-122.403223, 37.794124]
[-122.40348, 37.795567]
[-122.419204, 37.775196]
[-122.416164, 37.77789]
[-122.4000032, 37.7983181]
[-122.40348, 37.795567]
[-122.4186, 37.7753]
[-122.416164, 37.77789]
[-122.391011, 37.777092]
[-122.389344, 37.783824]
[-122.347745, 37.580093]
[-122.3999719, 37.7870923]
[-122.40348, 37.795567]
[-122.401024, 37.764395]
[-122.410305, 37.76857]
[-122.4194155, 37.7749295]
[-122.416164, 37.77789]
[-122.404403, 37.78691]
[-122.40348, 37.795567]
[-122.408646, 37.784137]
[-122.416942, 37.781359]
[-122.402683, 37.771706]
[-122.410305, 37.76857]
[-122.420572, 37.761404]
[-122.421752, 37.760867]
[-122.391236, 37.779624]
[-122.389344, 37.783824]
[-122.4035368, 37.7875973]
[-122.40348, 37.795567]
[-122.400591, 37.782163]
[-122.38

In [328]:
list_nearest_vegan_coor

[[-122.405173, 37.795717],
 [-122.416164, 37.77789],
 [-122.405173, 37.795717],
 [-122.419145, 37.761621],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.419145, 37.761621],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.416942, 37.781359],
 [-122.416164, 37.77789],
 [-122.416164, 37.77789],
 [-122.416164, 37.77789],
 [-122.419145, 37.761621],
 [-122.405173, 37.795717],
 [-122.419145, 37.761621],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.416164, 37.77789],
 [-122.405173, 37.795717],
 [-122.416164, 37.77789],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.416942, 37.781359],
 [-122.405173, 37.795717],
 [-122.416164, 37.77789],
 [-122.405173, 37.795717],
 [-122.419145, 37.761621],
 [-122.416942, 37.781359],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.405173, 37.795717],
 [-122.421604, 37.768066],
 [-122.4

In [409]:
dist

[1451.608,
 661.631,
 780.205,
 574.959,
 877.925,
 1331.007,
 529.778,
 943.676,
 428.327,
 793.612,
 362.754,
 1052.091,
 1220.925,
 401.401,
 401.401,
 1274.213,
 1535.399,
 300.984,
 368.599,
 574.959,
 808.962,
 991.511,
 401.401,
 366.55,
 227.879,
 352.008,
 494.812,
 241.724,
 860.137,
 924.787,
 978.958,
 789.762,
 574.959,
 851.178,
 401.401,
 444.45,
 495.736,
 877.336,
 565.158,
 661.631,
 574.959,
 517.681,
 294.757,
 454.52,
 117.571,
 764.897,
 808.118,
 569.711,
 924.978,
 895.433,
 840.772,
 954.406,
 887.557,
 309.686,
 563.413,
 654.766,
 727.182,
 401.401,
 812.539,
 556.372,
 43.021,
 621.805,
 995.164,
 725.377,
 563.413,
 401.401,
 883.976,
 981.962,
 882.366,
 423.981,
 357.596,
 306.369,
 995.164,
 607.571,
 821.088,
 253.237,
 828.469,
 401.401,
 537.984,
 304.777,
 777.945,
 957.051,
 541.773,
 914.556,
 401.401,
 304.777,
 1237.668,
 1537.331,
 401.401,
 73.195,
 1178.543,
 665.824,
 1882.021,
 624.325,
 401.401,
 401.401,
 13.653,
 732.189,
 404.976,
 791.8

In [329]:
list_nearest_vegan

['Enjoy Vegetarian Restaurant',
 'Ananda Fuara',
 'Enjoy Vegetarian Restaurant',
 'Gracias Madre',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Gracias Madre',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Golden Era Vegan Restaurant',
 'Ananda Fuara',
 'Ananda Fuara',
 'Ananda Fuara',
 'Gracias Madre',
 'Enjoy Vegetarian Restaurant',
 'Gracias Madre',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Ananda Fuara',
 'Enjoy Vegetarian Restaurant',
 'Ananda Fuara',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Golden Era Vegan Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Ananda Fuara',
 'Enjoy Vegetarian Restaurant',
 'Gracias Madre',
 'Golden Era Vegan Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Enjoy Vegetarian Restaurant',
 'Shizen',
 'Enjoy Vege

In [ ]:
nueva columna = distancia minima (empresa - parque)

In [117]:
df2= pd.DataFrame(new_list_vegan)
df2["name"] = "Vegan Food"
df2

,name,lat,lon,type,distance
0,Vegan Food,37.761621,-122.419145,"{'typepoint': {'type': 'Point', 'coordinates':...",2467
1,Vegan Food,37.768066,-122.421604,"{'typepoint': {'type': 'Point', 'coordinates':...",2152
2,Vegan Food,37.760760,-122.503997,"{'typepoint': {'type': 'Point', 'coordinates':...",5142
3,Vegan Food,37.777890,-122.416164,"{'typepoint': {'type': 'Point', 'coordinates':...",2841
4,Vegan Food,37.770198,-122.446670,"{'typepoint': {'type': 'Point', 'coordinates':...",278
5,Vegan Food,37.807002,-122.432295,"{'typepoint': {'type': 'Point', 'coordinates':...",4510
6,Vegan Food,37.847406,-122.251847,"{'typepoint': {'type': 'Point', 'coordinates':...",19220
7,Vegan Food,37.760867,-122.421752,"{'typepoint': {'type': 'Point', 'coordinates':...",2289
8,Vegan Food,37.795717,-122.405173,"{'typepoint': {'type': 'Point', 'coordinates':...",4772
9,Vegan Food,37.781359,-122.416942,"{'typepoint': {'type': 'Point', 'coordinates':...",2990


In [120]:
#1. MARKER without icon
for index, row in df2.iterrows():
    vegan = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}

#2. Icon
    icon = Icon (color="lightgray", opacity = 0.6, prefix = "fa", icon="pagelines", icon_color = "green")    
    
#3. Marker
    new_marker = Marker(**vegan, icon = icon, radius = 2)

#4. Add the Marker
    new_marker.add_to(sanFran_map)

In [121]:
sanFran_map

In [209]:

df_SanF.reset_index(inplace = True, drop=True)
df_SanF.drop(['index'], axis=1)

,level_0,name,number_of_employees,total_money_raised,Country,City,lat,lon
0,0,Digg,60,$45M,USA,San Francisco,37.764726,-122.394523
1,1,Scribd,50,$25.8M,USA,San Francisco,37.789634,-122.404052
2,2,Powerset,60,$22.5M,USA,San Francisco,37.778613,-122.395289
3,3,Technorati,35,$32.1M,USA,San Francisco,37.779558,-122.393041
4,4,Kyte,40,$23.4M,USA,San Francisco,37.788482,-122.409173
...,...,...,...,...,...,...,...,...
315,315,UTOPY,40,$7.5M,USA,San Francisco,37.767575,-122.411157
316,316,ZoomSystems,100,$87M,USA,San Francisco,37.781689,-122.391061
317,317,Mondo Media,9,$0,USA,San Francisco,37.764169,-122.402021
318,318,TwtBks,3,$25k,USA,San Francisco,37.761102,-122.416001


In [191]:
df_SanF

,name,number_of_employees,total_money_raised,Country,City,lat,lon
1,Digg,60,$45M,USA,San Francisco,37.764726,-122.394523
4,Scribd,50,$25.8M,USA,San Francisco,37.789634,-122.404052
8,Powerset,60,$22.5M,USA,San Francisco,37.778613,-122.395289
9,Technorati,35,$32.1M,USA,San Francisco,37.779558,-122.393041
12,Kyte,40,$23.4M,USA,San Francisco,37.788482,-122.409173
...,...,...,...,...,...,...,...
7776,UTOPY,40,$7.5M,USA,San Francisco,37.767575,-122.411157
7821,ZoomSystems,100,$87M,USA,San Francisco,37.781689,-122.391061
7879,Mondo Media,9,$0,USA,San Francisco,37.764169,-122.402021
7902,TwtBks,3,$25k,USA,San Francisco,37.761102,-122.416001


In [ ]:
haversine(coord1, coord2)

# QUERY = pet grooming

In [372]:
query_3 = "pet grooming"

In [373]:
# Doing the same thing with limit=20

all_pet = get_results_from_foursquare (query_3, sanFran_location, limit=20)

In [374]:
new_list_pet = []
for i in all_pet["results"]:
    
    name = i["name"]
    address = i["location"]["formatted_address"]
    lon = i["geocodes"]["main"]["latitude"]
    lat = i["geocodes"]["main"]["longitude"]
    distance = i['distance']
    
    
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_pet.append({"name":name, "lat":lat, "lon":lon, "type":type_, "distance":distance})
    
new_list_pet[:2]

[{'name': 'The Refinery Grooming Club',
  'lat': -122.41851,
  'lon': 37.755723,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [-122.41851, 37.755723]}},
  'distance': 2735},
 {'name': "Bernie's Grooming City Dogs",
  'lat': -122.389344,
  'lon': 37.783824,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [-122.389344, 37.783824]}},
  'distance': 5314}]

In [376]:
pet = db.get_collection("pet")
pet

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'pet')

In [ ]:
pet.insert_many(new_list_pet)

In [381]:
pet.find_one()

{'_id': ObjectId('635ffa5e4f45b207b5cf1a32'),
 'name': 'The Refinery Grooming Club',
 'lat': -122.41851,
 'lon': 37.755723,
 'type': {'typepoint': {'type': 'Point',
   'coordinates': [-122.41851, 37.755723]}},
 'distance': 2735}

In [193]:
def type_point (list_):
    return {"type": "Point", "coordinates": list_}

In [383]:
list_nearest_pet=[]
list_nearest_pet_coor=[]

for index, rows in df_SanF.iterrows():
    try:
        lat =  df_SanF['lat'][index]
        lon = df_SanF['lon'][index]
    
        query = {"type.typepoint": {
                "$near": {
                    "$geometry": {'type': 'Point', 'coordinates':[lon, lat]}, "$maxDistance": 2000
                }}       
            }
        list_ = list(pet.find(query))
        v1 = list_[0]['name']
        v2 = list_[0]['type']['typepoint']['coordinates']
        
        list_nearest_pet.append(v1)
        list_nearest_pet_coor.append(v2)
        
    except:
        lat = 0
        lon = 0
    
        query = {"type.typepoint": {
                "$near": {
                    "$geometry": {'type': 'Point', 'coordinates':[lon, lat]}, "$maxDistance": 2000
                }}       
            }
        
        
    

In [384]:
list_nearest_pet_coor

[[-122.410305, 37.76857],
 [-122.40348, 37.795567],
 [-122.389344, 37.783824],
 [-122.389344, 37.783824],
 [-122.40348, 37.795567],
 [-122.410305, 37.76857],
 [-122.389344, 37.783824],
 [-122.40348, 37.795567],
 [-122.41851, 37.755723],
 [-122.389344, 37.783824],
 [-122.389344, 37.783824],
 [-122.389344, 37.783824],
 [-122.389344, 37.783824],
 [-122.430161, 37.771485],
 [-122.430161, 37.771485],
 [-122.410305, 37.76857],
 [-122.410305, 37.76857],
 [-122.389344, 37.783824],
 [-122.410305, 37.76857],
 [-122.389344, 37.783824],
 [-122.40348, 37.795567],
 [-122.40348, 37.795567],
 [-122.430161, 37.771485],
 [-122.40348, 37.795567],
 [-122.410305, 37.76857],
 [-122.40348, 37.795567],
 [-122.389344, 37.783824],
 [-122.40348, 37.795567],
 [-122.40348, 37.795567],
 [-122.389344, 37.783824],
 [-122.40348, 37.795567],
 [-122.410305, 37.76857],
 [-122.389344, 37.783824],
 [-122.40348, 37.795567],
 [-122.430161, 37.771485],
 [-122.389344, 37.783824],
 [-122.389344, 37.783824],
 [-122.410305, 37.76

In [385]:
list_nearest_pet

["Bernie's Grooming",
 'Nova Express Personal Grooming Services',
 "Bernie's Grooming City Dogs",
 "Bernie's Grooming City Dogs",
 'Nova Express Personal Grooming Services',
 "Bernie's Grooming",
 "Bernie's Grooming City Dogs",
 'Nova Express Personal Grooming Services',
 'The Refinery Grooming Club',
 "Bernie's Grooming City Dogs",
 "Bernie's Grooming City Dogs",
 "Bernie's Grooming City Dogs",
 "Bernie's Grooming City Dogs",
 'San Francisco Pet Grooming',
 'San Francisco Pet Grooming',
 "Bernie's Grooming",
 "Bernie's Grooming",
 "Bernie's Grooming City Dogs",
 "Bernie's Grooming",
 "Bernie's Grooming City Dogs",
 'Nova Express Personal Grooming Services',
 'Nova Express Personal Grooming Services',
 'San Francisco Pet Grooming',
 'Nova Express Personal Grooming Services',
 "Bernie's Grooming",
 'Nova Express Personal Grooming Services',
 "Bernie's Grooming City Dogs",
 'Nova Express Personal Grooming Services',
 'Nova Express Personal Grooming Services',
 "Bernie's Grooming City Dog